### Downloading the data

In [2]:
# Imports

import os
import zipfile
import requests
import json
import pandas as pd
import dask.dataframe as dd
import pyarrow.feather as feather
from urllib.request import urlretrieve

In [3]:
%load_ext memory_profiler
%load_ext rpy2.ipython

In [5]:
# Necessary constants

article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/"
combined_data_path = f"{output_directory}/combined_data.csv"

In [22]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [6]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

### Peak at one of the csv files



In [7]:
df = pd.read_csv("../data/MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv")

In [8]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13


In [9]:
df.shape

(966420, 6)

In [10]:
# Define a skeleton for concatenating data frames

combined_df = pd.DataFrame({
    "time": [], 
    "lat_min": [], 
    "lat_max": [], 
    "lon_min": [], 
    "lon_max": [], 
    "rain (mm/day)": [], 
    "model": []
})

### Concatenate all CSV files

In [11]:
%%time

exclude_files = ['observed_daily_rainfall_SYD.csv' , 'combined_data.csv']
for filename in os.listdir(output_directory):
 
    if filename.endswith('.csv'):
        if filename in exclude_files:
            continue
        else:
            model = filename.partition('_daily_rainfall')[0]
            df = pd.read_csv(output_directory + filename)
            df["model"] = model
            combined_df = pd.concat([combined_df, df], axis=0)


CPU times: user 46.1 s, sys: 21.1 s, total: 1min 7s
Wall time: 1min 13s


### Check if all csv files are incorporated as model column

In [16]:
# Sanity check - there should be 27 different models

assert len(list(combined_df['model'].value_counts()))==27, "Wrong number of files concatenated"
combined_df['model'].value_counts()

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

### Store the combined df to a csv file

In [27]:
%%time

combined_df.to_csv(combined_data_path)

CPU times: user 6min 16s, sys: 6.4 s, total: 6min 23s
Wall time: 6min 29s


In [ ]:
%%sh

du -sh ../data/combined_data.csv

6.0G	../data/combined_data.csv


**Note**: Time taken is calculated by combining the wall times of data concatenation and storing the data frame as a CSV file.

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |Mac OS Monterey   |16GB    |     M1 |  Yes   |   9min 13s |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |   9min 21s |
| Navya       |Mac OS X (Big Sur)|16 GB|     M1    |  Yes   |  6min 29s  |
| Nobby       |Mac OS X (Big Sur)|8 GB |M1         |   Yes  |  6min 34s  |

### 3. Load the combined CSV to memory and perform a simple EDA

#### 3.1 Loading the entire data frame (Reference)

Recording the runtime and memory usage of loading and performing a simple EDA [`value_counts()`] on the whole dataset using Pandas as reference for other approaches.

In [ ]:
%%time
%%memit

rainfall_df = pd.read_csv(combined_data_path)
print(rainfall_df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 26990.42 MiB, increment: 1974.73 MiB
CPU times: user 48 s, sys: 14.4 s, total: 1min 2s
Wall time: 1min 12s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |Mac OS Monterey   |16GB    |     M1 |  Yes   |   1min 38s |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  | 2min 52s   |
| Navya       |Mac OS X (Big Sur)| 16 GB|    M1    |  Yes   |   1min 12 s|
| Nobby       |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  | 1min 40s   |            

#### 3.2 Approach 1: Changing `dtype` of the data

Modifying data types for the features to optimize the memory usage and checking if this improves the run time when compared to baseline.

In [ ]:
%%time
%%memit

print(f"Memory usage with float64: {rainfall_df.memory_usage().sum() / 1e6:.2f} MB")
rainfall_df = rainfall_df.astype(
    dtype={
        "lat_min": "float32",
        "lat_max": "float32",
        "lon_min": "float32",
        "lon_max": "float32",
        "rain (mm/day)": "float32",
        "time": "datetime64[s]"
    },
    errors='ignore')
print(f"Memory usage with float32: {rainfall_df.memory_usage().sum() / 1e6:.2f} MB")
print(rainfall_df["model"].value_counts())

Memory usage with float64: 3997.94 MB
Memory usage with float32: 2748.59 MB
MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 26533.81 MiB, increment: 10435.67 MiB
CPU times: user 14.9 s, sys: 4.17 s, total: 19 s
Wall time: 20.8 s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |Mac OS Monterey   |16GB    |     M1 |  Yes   |   27s |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |   45.3s    |
| Navya       |Mac OS X (Big Sur)|16 GB |   M1     |   Yes  |  20.8s    |
| Nobby       |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  | 34.9s.     |     

#### 3.3 Approach 2: Load data in chunks

Loading the data in chunks of size 10 millions instead of loading all at once and checking if this improves the run time and memory usage when compared to baseline.

In [ ]:
%%time
%%memit

counts = pd.Series(dtype=int)
for chunk in pd.read_csv(combined_data_path, chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)

print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 22771.11 MiB, increment: 8369.58 MiB
CPU times: user 47 s, sys: 4.93 s, total: 51.9 s
Wall time: 53.4 s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |Mac OS Monterey   |16GB    |     M1 |  Yes   |   1min 13s |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |   59s      |
| Navya       |Mac OS X (Big Sur)|16 GB|   M1      |   Yes  |   53.4s    |
| Nobby       |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  | 1min 12s   |            

#### 3.4 Approach 3: Dask

[Dask](https://dask.org/) is a data analysis library similar to Pandas but supports parallelism and elegantly handles out of memory datasets. Using it to check for any improvements in the runtime and memory usage.

In [ ]:
%%time
%%memit

rainfall_ddf = dd.read_csv(combined_data_path)
print(rainfall_ddf["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 12826.03 MiB, increment: 2355.45 MiB
CPU times: user 33.6 s, sys: 3.63 s, total: 37.3 s
Wall time: 12.4 s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |Mac OS Monterey   |16GB    |     M1 |  Yes   |   19s |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |   23.3s    |
| Navya       |Mac OS X (Big Sur)|16 GB|   M1      |   Yes  |   12.4s    |               
| Nobby       |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  | 17.8s      |

**Observations:**

1. The baseline takes the maximum time to load the dataset and perform a simple EDA when compared to other approaches. Also, it can be seen loading the entire dataset at once in memory intensive.
2. In **Approach 1**, we inspected the memory usage with and without data type optimization. As you can see there is a significant improvement in memory usage from 3997.94 MB to 2748.59 MB when the data types for the features are optimized. The intention of this step is to not compare the runtime to other approaches but to check for improvements in memory usage.
3. In **Approach 2**, we tried optimizing the runtime by loading data in chunks of size 10 million iteratively along with performing EDA. It can be observed that the runtime has improved by a large margin from around 2 mins 52 seconds to 59 seconds and there is also a significant improvement in terms of memory usage reducing the peak usage from 5864.23 MiB to 2356.17 MiB.
4. In **Approach 3**, we used a third-party library called Dask to improve the performance using parallelism and efficient handling of the large dataset. This approach is the quickest in terms of runtime among all other approaches taking only 23 seconds to complete the whole operation. But the peak memory usage is not as efficient as Approach 1 or Approach 2.

> To summarise, the Dask approach seems to provide the best runtime while the approach of loading the dataset in chunks provides the optimal memory usage.

### 4. Simple EDA in `R`

#### 4.1 Create feather file

In [ ]:
%%time
%%memit

rainfall_df.to_feather('../data/combined_data_feather.feather')

peak memory: 13741.22 MiB, increment: 6503.16 MiB
CPU times: user 5.06 s, sys: 2.54 s, total: 7.59 s
Wall time: 5.85 s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |                  |     |           |        |            |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |            |
| Navya       |Mac OS X (Big Sur)|16 GB|   M1      |   Yes  |  5.85s    |               
| Nobby       |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |     |

##### Check the file size

In [ ]:
%%sh
du -sh ../data/combined_data.csv
du -sh ../data/combined_data_feather.feather

6.0G	../data/combined_data.csv
1.2G	../data/combined_data_feather.feather


#### 4.2 EDA

In [ ]:
%%time
%%R
library(dplyr)
library(arrow)
start_time <- Sys.time()

rainfall_df_r <- arrow::read_feather("../data/combined_data_feather.feather")
result <- rainfall_df_r |>  count(model)

end_time <- Sys.time()
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 11.10052 secs
CPU times: user 6.27 s, sys: 7.4 s, total: 13.7 s
Wall time: 15 s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |                  |     |           |        |            |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |            |
| Navya       |Mac OS X (Big Sur)|16 GB|   M1      |   Yes  |  11.100s    |               
| Nobby       |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |     |

In [ ]:
%%time
%%R
start_time <- Sys.time()

print(class(rainfall_df_r))
glimpse(rainfall_df_r)

end_time <- Sys.time()
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"
Rows: 62,467,843
Columns: 8
$ `Unnamed: 0`    <int> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, …
$ time            <dttm> 1889-01-01 04:00:00, 1889-01-02 04:00:00, 1889-01-03 …
$ lat_min         <dbl> -35.43987, -35.43987, -35.43987, -35.43987, -35.43987,…
$ lat_max         <dbl> -33.57462, -33.57462, -33.57462, -33.57462, -33.57462,…
$ lon_min         <dbl> 141.5625, 141.5625, 141.5625, 141.5625, 141.5625, 141.…
$ lon_max         <dbl> 143.4375, 143.4375, 143.4375, 143.4375, 143.4375, 143.…
$ `rain (mm/day)` <dbl> 4.244226e-13, 4.217326e-13, 4.498125e-13, 4.251282e-13…
$ model           <chr> "MPI-ESM-1-2-HAM", "MPI-ESM-1-2-HAM", "MPI-ESM-1-2-HAM…
Time difference of 0.01221609 secs
CPU times: user 16.5 ms, sys: 5.34 ms, total: 21.8 ms
Wall time: 21.4 ms


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Cici        |                  |     |           |        |            |
| Gautham     |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |            |
| Navya       |Mac OS X (Big Sur)|16 GB|   M1      |   Yes  | 0.12s    |               
| Nobby       |Mac OS X (Big Sur)|8 GB |   M1      |   Yes  |     |

#### 4.3 Reason to choose feather file

- Feathers files are about 30% of the size as the CSV file. In this case CSV file is almost 6GB, while feather file is only 1.2GB
- Saving the data in feather file takes less than 5 minutes and loading the data takes less than 20 seconds.
- Pandas exchange was not choosen because it is slow, Arrow was not selected because it is more suitable for 
- Parquet file could have compressed the data further and we can save some more space, but in this case we are satisfied with the size 1.2GB for such a big dataset. 